#이미지 픽셀 줄이기

In [ ]:
!pip3 install Pillow

In [ ]:
import os
from PIL import Image

In [ ]:
#챗 지피티 사용
#픽셀 크기가 40000000 이상인 사진들 픽셀 크기 줄이는 함수
# DecompressionBombWarning 경고를 무시하거나 설정 변경
Image.MAX_IMAGE_PIXELS = None  # 무제한으로 설정하거나 원하는 최대 픽셀 수로 설정

def resize_image(image_path, max_pixels=40000000):
    with Image.open(image_path) as img:
        original_width, original_height = img.size
        original_pixels = original_width * original_height

        if original_pixels > max_pixels:
            scaling_factor = (max_pixels / original_pixels) ** 0.5
            new_width = int(original_width * scaling_factor)
            new_height = int(original_height * scaling_factor)

            img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            img.save(image_path)
            print(f"{image_path}: Image resized to {new_width}x{new_height} pixels.")
        else:
            print(f"{image_path}: Image size is within the limit. No resizing needed.")

In [ ]:
def process_images_in_folder(folder_path, max_pixels=40000000):
    # 이미지 파일 확장자 리스트
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']

    # 폴더 안의 파일 목록 가져오기
    files = os.listdir(folder_path)

    # 이미지 파일만 필터링
    image_files = [file for file in files if os.path.splitext(file)[1].lower() in image_extensions]

    print("Processing image files in the folder:")
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        resize_image(image_path, max_pixels)

In [ ]:
# 예시 사용법:
folder_path = '/Users/jihoonlee/Desktop/ASAC/ML/MLproject/poster_4' # 사진들이 있는 파일 경로
process_images_in_folder(folder_path)

#구글 비전 api 연동 및 전처리

In [ ]:
!pip install opencv-contrib-python
!pip install --upgrade google-cloud-vision
!pip install utils

In [ ]:
import os
import io
import re
import math

import numpy as np
import pandas as pd

from PIL import ImageFont, ImageDraw, Image
import platform
import matplotlib as plt
# from utils import plt_imshow
from PIL import Image
import cv2
from google.cloud import vision

In [ ]:
#구글 api 불러오기 설정 -> 구글에서 그대로 복사
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)

    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []

            for i in range(len(img)):
                titles.append(title)

        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)

            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])

        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

In [ ]:
#구글 api 불러오기 설정 -> 폰트, 글꼴, 컬러 -> 구글에서 그대로 복사
def putText(image, text, x, y, color=(0, 255, 0), font_size=22):
    if type(image) == np.ndarray:
        color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(color_coverted)

    if platform.system() == 'Darwin':
        font = 'AppleGothic.ttf'
    elif platform.system() == 'Windows':
        font = 'malgun.ttf'
    else:
        font = 'NanumGothic.ttf'

    image_font = ImageFont.truetype(font, font_size)
    font = ImageFont.load_default()
    draw = ImageDraw.Draw(image)

    draw.text((x, y), text, font=image_font, fill=color)

    numpy_image = np.array(image)
    opencv_image = cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

    return opencv_image

In [ ]:
# 임의의 사각형을 이루는 각 변의 벡터 계산-> title_rate 계산을 위해 필요

def calculate_polygon_area(vertices):
    vectors = []

    for i in range(len(vertices)):
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i + 1) % len(vertices)]
        vectors.append((x2 - x1, y2 - y1))

    # 외적을 통한 넓이 계산
    area = 0
    for i in range(len(vectors)):
        area += vectors[i][0] * vectors[(i + 1) % len(vectors)][1]
        area -= vectors[i][1] * vectors[(i + 1) % len(vectors)][0]

    area = abs(area) / 2
    return area

In [ ]:
# 이미지 크기 계산 : 경로 설정 필요함!
def size(img):
    im = Image.open('/Users/jihoonlee/Desktop/ASAC/ML/'+img) # 여기
    width, height = im.size

    return width*height

In [ ]:
# 특수문자 제거 정규식 (안 썼음)
def remove_special_characters(text):
    result = re.sub(r'[^\w\s]', '-', text)
    result = re.sub(r'[-\*]', '', text)

    return result


In [ ]:
# 구글 클라우드 연동!
# 키 값 입력 필요함!

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/jihoonlee/Desktop/ASAC/ML/MLproject/service_secret_key.json" # 키

client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [ ]:
def make_list(directory,moive_idx,image_index):
    # movie_idx -> 영화순서
    lst=[]

    for image_name, index in image_index.items():
        print("<",index,image_name,">")
        path=directory+"/"+image_name

        with io.open(path, 'rb') as image_file:
            content = image_file.read()

        client = vision.ImageAnnotatorClient()

        # 이미지 컨텐츠 설정
        image = vision.Image(content=content)

        # 이미지 컨텍스트 설정 (언어 힌트 추가)
        image_context = vision.ImageContext(language_hints=["ko"])

        # 텍스트 감지 요청
        response = client.text_detection(image=image, image_context=image_context)
        texts = response.text_annotations

            # texts 에서 글자 이상한 거 여기서 없애기!
            # texts = remove_special_characters(texts[0].description)

        poster_size = size(image_name)




        text_rate1=0 #글자별 비율
        for idx, text in enumerate(texts):
            vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]
            box_size = calculate_polygon_area(vertices)

            temp = []

            temp.append(moive_idx) #영화 인덱스
            temp.append(os.path.splitext(image_name)[0]) # 영화 이름
            #temp.append(poster_size) # 포스터 전체 사이즈
            #temp.append(idx)
            #temp.append(text.description) #인식한 단어
            #temp.append(len(text.description)) #인식한 단어 전체 길이(한 음절 기준)
            #temp.append(vertices[0]) #좌표 좌측 하단
            #temp.append(vertices[1]) #좌표 우측 하단
            #temp.append(vertices[2]) #좌표 우측 상단
            #temp.append(vertices[3]) #좌표 좌측 상단
            #temp.append(calculate_polygon_area(vertices)) #단어 상자 크기_ 글자 크기로 간주

            if idx!=0:
                text_rate1+=box_size/poster_size
            temp.append(text_rate1) # 전체 포스터 대비 단어 상자별 비율



        lst.append(temp)
        moive_idx +=1

    return lst

#         맥에서 안될 때 사용해보기
# for text in texts:
#     print('\n"{}"'.format(text.description))
#     vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]
#     print('텍스트 위치: {}'.format(vertices))


In [ ]:
# 경로 설정 후 포스터 리스트 받기
directory = "/Users/jihoonlee/Desktop/ASAC/ML/" # 여기
files = sorted(os.listdir(directory))
image_index = {}

# # 파일 목록을 순회하며 이미지 파일인지 확인하고 인덱스 부여
for index, file in enumerate(files):
    file_extension = os.path.splitext(file)[1].lower()
    if file_extension in ['.jpg']:
        image_index[file] = index

In [ ]:
# 1 => moive_idx 시작점
lst= make_list(directory,1,image_index)

data = pd.DataFrame(lst,
                   columns = ["movie_idx","Image_name", "text_rate"])

#csv 저장경로
data.to_csv('/Users/jihoonlee/Desktop/ASAC/ML/final.csv', header=False, sep=',',encoding="utf-8") #여기

#lst_2 = make_list(image_index,math.floor(len(image_index)/2),math.floor(len(image_index)/2)+1) #test용

In [ ]:
#결과 확인
data

# 구글비전api데이터, box데이터 조인

In [ ]:
path1 = "/Users/jihoonlee/Desktop/ASAC/ML/MLproject/구글비전api.csv" # 구글비전api데이터
df1 = pd.read_csv(path, encoding = 'utf-8')

path2 = "/Users/jihoonlee/Desktop/ASAC/ML/MLproject/box데이터.csv" # box데이터
df2 = pd.read_csv(path2, encoding = 'cp949')

In [ ]:
merged_df = pd.merge(df1, df2, on='Image_name', how='inner') #Image_name으로 이너조인

In [ ]:
merged_df.to_csv("/Users/jihoonlee/Desktop/ASAC/ML/MLproject/join데이터.csv",index = False,encoding='utf-8',) # join데이터.csv 저장 -> 경로 변경 필요

# Box 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np

path_1 = ("/Users/jihoonlee/Desktop/ASAC/ML/final2.csv") # join데이터.csv 경로

df_1 = pd.read_csv(path_1)#텍스트 인식파일

In [ ]:
# 이미지의 가로, 세로 길이를 받아 3,4등분의 좌표 저장 함수
def divide_image(image_width, image_height):
    # 각 구역의 가로와 세로 크기를 계산
    segment_width = image_width // 3
    segment_height = image_height // 4

    segments = []
    # 각 구역을 순회하면서 각 구역의 좌표를 저장
    for row in range(4):
        for col in range(3):
            left = col * segment_width
            upper = (3 - row) * segment_height  # y 좌표 반전 -> 박스데이터의 좌표는 위에서 아래로 커지므로 y좌표 반전 필요
            right = left + segment_width
            lower = upper + segment_height
            segments.append(((left, upper), (right, lower)))
    return segments

In [ ]:
#박스의 좌표와 면적 10%가 넘어가는 구역 계산 함수
def find_rectangle_segments(image_width, image_height, rect_left, rect_top, rect_right, rect_bottom):
    # 각 구역의 좌표를 가져오기
    segments = divide_image(image_width, image_height)

    # 직사각형의 좌표와 넓이를 계산
    rect_width = rect_right - rect_left
    rect_height = rect_bottom - rect_top
    rect_area = rect_width * rect_height

    # 결과를 저장할 리스트
    valid_segments = []

     # 각 구역을 순회하면서 직사각형이 해당 구역의 넓이의 10% 이상 차지하는지 확인
    for i, ((left, upper), (right, lower)) in enumerate(segments, start=1):
        # 직사각형과 구역의 교차 영역 넓이를 계산
        inter_left = max(rect_left, left)
        inter_upper = max(rect_top, upper)
        inter_right = min(rect_right, right)
        inter_lower = min(rect_bottom, lower)

        # 교차 영역이 존재하는 경우
        if inter_left < inter_right and inter_upper < inter_lower:
            inter_area = (inter_right - inter_left) * (inter_lower - inter_upper)
            # 교차 영역의 넓이가 구역의 넓이의 10% 이상인 경우 해당 구역을 저장
            if inter_area >= 0.1 * ((right - left) * (lower - upper)):
                valid_segments.append(i)

    return valid_segments

In [ ]:
#데이터 프레임에 넣을 결과 리스트 선언
total_data=[]

In [ ]:
#total_data 에 결과값 넣기
for i in (df_1.image_name.unique()):
    image_count=df_1[df_1["image_name"]==i] # 1개의 영화에 박스가 여러개 있을 수 있으므로 각 영화마다 따로따로 데이터 프레임 추출

    image_width = image_count.iat[0, 8]
    image_height = image_count.iat[0, 9]
    image_size = image_width * image_height
    regions = divide_image(image_width, image_height) # 이미지의 가로, 세로 길이를 받아 3,4등분의 좌표 저장 함수
    title_rate=0 #박스 비율들의 합 -> box_rate
    text_rate=0 # 이미 join데이터.csv에 있지만 제목을 인식 못했을경우 sum_rated와 합치기 위해 변수 선언 -> 밑에 나옴
    occupied_regions = []
    # 다음 영화로 넘어가면 구역들 초기화
    part_1 = 0
    part_2 = 0
    part_3 = 0
    part_4 = 0
    part_5 = 0
    part_6 = 0
    part_7 = 0
    part_8 = 0
    part_9 = 0
    part_10=0
    part_11=0
    part_12=0

    a=[] #box들의 part를 저장하는 리스트 선언

    # 한 영화의 박스 한개씩 순회하여 총 title_rate, part 계산
    for j in range (len(image_count)):
        box_width = image_count.iat[j, 9]
        box_height = image_count.iat[j, 10]
        box_size = box_width * box_height #박스 넓이
        title_rate += (box_size / image_size)

        # box좌측상단 좌표
        x_1=image_count.iat[j,4]
        y_1=image_count.iat[j,5]

        # box 우측하단 좌표
        x_2 = x_1 + box_width
        y_2=y_1+box_height


        segments = find_rectangle_segments(image_width, image_height, x_1, y_1, x_2, y_2) #면적 10%가 넘어가는 구역 계산 함수
        # 아무 구역도 속하지 못하는 box가 있는지 확인
        if segments:
            print(f"직사각형이 10% 이상 속한 구역들: {segments}",i)
            pass
        else:
            print("직사각형이 어느 구역에도 10% 이상 속하지 않습니다.",i)
        for w in range(len(segments)):
            a.append(segments[w])

    # 리스트에 있는 part들 확인해서 원핫인코딩
    for z in range(len(a)):
        if a[z] == 1:
            part_1 = 1
        elif a[z] == 2:
            part_2 = 1
        elif a[z] == 3:
            part_3 = 1
        elif a[z] == 4:
            part_4 = 1
        elif a[z] == 5:
            part_5 = 1
        elif a[z] == 6:
            part_6 = 1
        elif a[z] == 7:
            part_7 = 1
        elif a[z] == 8:
            part_8 = 1
        elif a[z] == 9:
            part_9 = 1
        elif a[z] == 10:
            part_10 = 1
        elif a[z] == 11:
            part_11 = 1
        else:
            part_12 = 1

    if image_count.iat[j,2]<=title_rate: # 만약 join 데이터의 text_rate가 title_rate 보다 작다면 제목을 인식 못한걸로 판단
        text_rate=image_count.iat[j,2]+title_rate # 그래서 text_rate에 tittle_rate를 더해 줌
    else:
        text_rate=image_count.iat[j,2]

    total_data.append([i,text_rate,title_rate,part_1,part_2,part_3,part_4,part_5,part_6,part_7,part_8,part_9,part_10,part_11,part_12])

In [ ]:
#데이터프레임 만들기
text_data=pd.DataFrame(
    data=total_data,
    columns=["Image_name","text_rate","title_rate","part_1","part_2","part_3","part_4","part_5","part_6","part_7","part_8","part_9","part_10","part_11","part_12"])
#csv 저장. 경로설정 필요
text_data.to_csv("/Users/jihoonlee/Desktop/ASAC/ML/final3.csv", header=False, sep=',')

# Box데이터 전처리 part 중점 버전

In [ ]:
import pandas as pd
import numpy as np

path_1 = ("/Users/jihoonlee/Desktop/ASAC/ML/final2.csv") # join데이터.csv 경로

df_1 = pd.read_csv(path_1)#텍스트 인식파일

In [ ]:
#데이터 프레임에 넣을 결과 리스트 선언
total_data=[]

In [ ]:
#total_data 에 결과값 넣기
for i in (df_1.image_name.unique()):
    image_count=df_1[df_1["image_name"]==i] # 1개의 영화에 박스가 여러개 있을 수 있으므로 각 영화마다 따로따로 데이터 프레임 추출

    image_width = image_count.iat[0, 9]
    image_height = image_count.iat[0, 10]
    image_size = image_width * image_height
    image_x = [] #3등분 한 x좌표를 저장하는 리스트
    image_y = [] #4등분 한 y좌표를 저장하는 리스트
    title_rate=0 #박스 비율들의 합 -> box_rate
    text_rate=0 # 이미 join데이터.csv에 있지만 제목을 인식 못했을경우 sum_rated와 합치기 위해 변수 선언 -> 밑에 나옴
    # 다음 영화로 넘어가면 구역들 초기화
    part_1 = 0
    part_2 = 0
    part_3 = 0
    part_4 = 0
    part_5 = 0
    part_6 = 0
    part_7 = 0
    part_8 = 0
    part_9 = 0
    part_10=0
    part_11=0
    part_12=0

    # 3등분 한 x좌표 계산 및 저장
    for k in range(1, 4):
        x_part=image_width/3
        image_x.append(x_part*k)

    # 3등분 한 x좌표 계산 및 저장
    for m in range(1,5):
        y_part = image_height / 4
        image_y.append(y_part * m)


    # 한 영화의 박스 한개씩 순회하여 총 title_rate, part 계산
    for j in range (len(image_count)):
        box_width = image_count.iat[j, 6]
        box_height = image_count.iat[j, 7]
        box_size = box_width * box_height #박스 넓이
        title_rate += (box_size / image_size)

        x_1=image_count.iat[j,4] #박스 좌측상단 x좌표
        y_1=image_count.iat[j,5] #박스 좌측상단 y좌표
        x_2 = x_1 + box_width    #박스 우측하단 x좌표
        y_2=y_1+box_height       #박스 우측하단 y좌표
        x_3=(x_2+x_1)/2          #박스 중점 x좌표
        y_3=(y_1+y_2)/2          #박스 중점 y좌표

       # x_3가 있는구역 먼저 확인 후 y_3가 있는 구역을 확인하여 part확정 -> 원핫인코딩
        if 0 <= x_3 < image_x[0]:
            if 0<=y_3<image_y[0]:
                part_10=1
            elif image_y[0] <= y_3 <image_y[1]:
                part_7=1
            elif image_y[1] <= y_3 <image_y[2]:
                part_4=1
            else:
                part_1=1

        elif image_x[0] <= x_3 < image_x[1]:
            if 0 <= y_3 < image_y[0]:
                part_11=1
            elif image_y[0] <= y_3 < image_y[1]:
                part_8=1
            elif image_y[1] <= y_3 <image_y[2]:
                part_5=1
            else:
                part_2=1

        else:
            if 0 <= y_3 < image_y[0]:
                part_12=1
            elif image_y[0] <= y_3 < image_y[1]:
                part_9=1
            elif image_y[1] <= y_3 <image_y[2]:
                part_6=1
            else:
                part_3=1


    if image_count.iat[j,0]<=title_rate: # 만약 join 데이터의 text_rate가 title_rate 보다 작다면 제목을 인식 못한걸로 판단
        text_rate=image_count.iat[j,2]+title_rate # 그래서 text_rate에 tittle_rate를 더해 줌
    else:
        text_rate=image_count.iat[j,2]

    total_data.append([i,text_rate,title_rate,part_1,part_2,part_3,part_4,part_5,part_6,part_7,part_8,part_9,part_10,part_11,part_12])

In [ ]:
#데이터프레임 만들기
text_data=pd.DataFrame(
    data=total_data,
    columns=["Image_name","text_rate","title_rate","part_1","part_2","part_3","part_4","part_5","part_6","part_7","part_8","part_9","part_10","part_11","part_12"])
#csv 저장. 경로설정 필요
text_data.to_csv("/Users/jihoonlee/Desktop/ASAC/ML/final3.csv", header=False, sep=',')

## box중점버전 라벨 인코딩


In [ ]:
# 한 영화의 박스가 최대 4개라고 가정
import pandas as pd

# csv 불러오기
path = "/Users/jihoonlee/Desktop/ASAC/ML/MLproject/전처리 최종.csv" #전처리 끝난 csv
df = pd.read_csv(path, encoding = 'cp949')
df_2=df.copy()
df_1=df_2.loc[:,"part_1":"part_12"] #part_1~part_12 열만 남긴 데이터프레임

In [ ]:
 # 라벨링 함수 정의
def label_rows(row):
    for i, col in enumerate(df.columns):
        if row[col] != 0:
            return i + 1  # 열의 인덱스 + 1을 라벨로 사용

In [ ]:
# 라벨링 열 추가
df['part_all'] = df_1.apply(label_rows, axis=1)
df

In [ ]:
#csv 저장. 경로설정 필요
text_data.to_csv("/Users/jihoonlee/Desktop/ASAC/ML/final3.csv", header=False, sep=',')